In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier

In [2]:
data = pd.read_csv('processed_data.csv')

In [3]:
data

,Unnamed: 0,X,flag,alon_week,alon_wend,child_bull,comp_house,comp_noint_bed_16,comp_int_bed_16,comp_week,...,talk_phon_wend,text_week,text_wend,tran_week,tran_wend,tv_week,tv_wend,weight_16,work_week,work_wend
0,1,1,synthetic,1-2 hours,1-2 hours,No,Yes,NaN,NaN,1-2 hours,...,Any at all,Less than 1 hour,Less than 1 hour,Less than 1 hour,Less than 1 hour,1-2 hours,1-2 hours,59.294132,3 or more hours,3 or more hours
1,2,2,synthetic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,3,synthetic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,4,synthetic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.812426,NaN,NaN
4,5,5,synthetic,1-2 hours,3 or more hours,No,NaN,Yes,NaN,1-2 hours,...,Not at all,Not at all,Not at all,Less than 1 hour,Less than 1 hour,1-2 hours,3 or more hours,62.270030,1-2 hours,1-2 hours
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13729,13730,13730,synthetic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13730,13731,13731,synthetic,1-2 hours,3 or more hours,No,NaN,NaN,NaN,1-2 hours,...,Not at all,Less than 1 hour,Less than 1 hour,Less than 1 hour,Less than 1 hour,3 or more hours,3 or more hours,NaN,1-2 hours,1-2 hours
13731,13732,13732,synthetic,1-2 hours,1-2 hours,No,NaN,NaN,NaN,1-2 hours,...,Any at all,1-2 hours,1-2 hours,1 or more hours,1 or more hours,3 or more hours,3 or more hours,58.075832,Less than 1 hour,Less than 1 hour
13732,13733,13733,synthetic,Less than 1 hour,1-2 hours,No,Yes,NaN,NaN,Less than 1 hour,...,Any at all,1-2 hours,1-2 hours,Less than 1 hour,Less than 1 hour,Not at all,Less than 1 hour,NaN,Less than 1 hour,1-2 hours


In [4]:
directly_related = ['comp_week', 'comp_wend', 'text_week', 'text_wend', 'tv_week', 'tv_wend']
targets_prim_diag = data[['prim_diag']]

r2_list = []

In [6]:
# Directly related
for x in directly_related:
    features = data[[x]].copy()
    features.loc[features[x] == 'Not at all', x] = 0
    features.loc[features[x] == 'Less than 1 hour', x] = 1
    features.loc[features[x] == '1-2 hours', x] = 2
    features.loc[features[x] == '3 or more hours', x] = 3
    
    if x == 'comp_week':
        output = features
    else:
        output[x] = features


In [8]:
output['prim_diag'] = targets_prim_diag

In [9]:
output

,comp_week,comp_wend,text_week,text_wend,tv_week,tv_wend,prim_diag
0,2,2,1,1,2,2,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,1,0,0,2,3,NaN
...,...,...,...,...,...,...,...
13729,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13730,2,2,1,1,3,3,NaN
13731,2,2,2,2,3,3,NaN
13732,1,2,2,2,0,1,0.0


In [10]:
output = output.dropna(axis=0)

In [11]:
output

,comp_week,comp_wend,text_week,text_wend,tv_week,tv_wend,prim_diag
0,2,2,1,1,2,2,0.0
5,2,1,1,1,2,3,0.0
8,3,3,3,1,2,2,5.0
18,1,2,1,1,1,2,0.0
24,2,3,0,0,2,2,0.0
...,...,...,...,...,...,...,...
13716,3,3,2,2,2,3,0.0
13720,2,3,1,1,1,2,0.0
13722,0,0,0,0,2,0,0.0
13725,2,3,1,1,1,2,11.0


In [12]:
X = output[output.columns.difference(['prim_diag'])]
y = output[['prim_diag']]

In [13]:
X

,comp_week,comp_wend,text_week,text_wend,tv_week,tv_wend
0,2,2,1,1,2,2
5,2,1,1,1,2,3
8,3,3,3,1,2,2
18,1,2,1,1,1,2
24,2,3,0,0,2,2
...,...,...,...,...,...,...
13716,3,3,2,2,2,3
13720,2,3,1,1,1,2
13722,0,0,0,0,2,0
13725,2,3,1,1,1,2


In [14]:
y

,prim_diag
0,0.0
5,0.0
8,5.0
18,0.0
24,0.0
...,...
13716,0.0
13720,0.0
13722,0.0
13725,11.0


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

In [26]:
# ANN -> Doesn't work

In [27]:
clf = MLPClassifier(random_state=1, max_iter=10000)

In [28]:
clf.fit(X_train, y_train.values.ravel())

MLPClassifier(max_iter=10000, random_state=1)

In [29]:
y_pred = clf.predict(X_test)

In [30]:
y_pred

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [31]:
clf_2 = MLPClassifier(random_state=1, activation='logistic', max_iter=10000)

In [32]:
clf_2.fit(X_train, y_train.values.ravel())

MLPClassifier(activation='logistic', max_iter=10000, random_state=1)

In [33]:
y_pred_2 = clf_2.predict(X_test)

In [34]:
y_pred_2

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [35]:
clf_3 = MLPClassifier(random_state=1, activation='logistic', solver='sgd', max_iter=10000)

In [36]:
clf_3.fit(X_train, y_train.values.ravel())

MLPClassifier(activation='logistic', max_iter=10000, random_state=1,
              solver='sgd')

In [37]:
y_pred_3 = clf_3.predict(X_test)

In [47]:
y = y.astype(int)

In [48]:
y

,prim_diag
0,0
5,0
8,5
18,0
24,0
...,...
13716,0
13720,0
13722,0
13725,11


In [38]:
y_pred_3

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [39]:
# Use XGBoost instead

In [51]:
xgboost = XGBClassifier(use_label_encoder=False)
xgboost.fit(X_train, y_train)

ValueError: The label must consist of integer labels of form 0, 1, 2, ..., [num_class - 1].